## Retriever

First, we index 3 blog posts.

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain_community.vectorstores.chroma import Chroma

# Используйте токен, полученный в личном кабинете из поля Авторизационные данные
_URL_PRO = "https://wmapi-ift.saluteai-pd.sberdevices.ru/v1/"

def get_giga():
    return GigaChat(
            base_url=_URL_PRO,
            credentials="credentials ..."
            )    


urls = [
    "https://habr.com/ru/companies/sberdevices/articles/794773/",
    "https://habr.com/ru/companies/sberdevices/articles/792660/",
    "https://habr.com/ru/companies/sberdevices/articles/777578/",
    "https://habr.com/ru/companies/sberbank/articles/773180/",
    "https://habr.com/ru/companies/sberdevices/articles/790470/"
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)


In [3]:

_URL_PLUS = "https://beta.saluteai.sberdevices.ru/v1"
giga_embeddings = GigaChatEmbeddings(base_url=_URL_PLUS,
    credentials="credentials ...")

# Add to vectorDB

vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=giga_embeddings
)

retriever = vectorstore.as_retriever()

Then we create a retriever tool.

In [69]:
from langchain.tools.retriever import create_retriever_tool

from langgraph.prebuilt import ToolExecutor

tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Выполняет поиск по статьям про GigaChat на Хабре",
)

tools = [tool]

tool_executor = ToolExecutor(tools)

## Agent state
 
We will defined a graph.

A `state` object that it passes around to each node.

Our state will be a list of `messages`.

Each node in our graph will append to it.

In [70]:
import operator
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

## Nodes and Edges

We can lay out an agentic RAG graph like this:

* The state is a set of messages
* Each node will update (append to) state
* Conditional edges decide which node to visit next


In [71]:
import operator
from typing import Annotated, Sequence, Type, TypedDict

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import GigaChat
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.utils.function_calling import convert_to_gigachat_function

from langgraph.prebuilt import ToolInvocation

### Edges


def should_retrieve(state: Type[AgentState]) -> str:
    """
    Decides whether the agent should retrieve more information or end the process.

    This function checks the last message in the state for a function call. If a function call is
    present, the process continues to retrieve information. Otherwise, it ends the process.

    Args:
        state (messages): The current state

    Returns:
        str: A decision to either "continue" the retrieval process or "end" it
    """
    
    print("---DECIDE TO RETRIEVE---")
    messages = state["messages"]
    last_message = messages[-1]


    if not isinstance(last_message, BaseMessage):
        raise Exception("Last message is not a BaseMessage instance")

    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        print("---DECISION: DO NOT RETRIEVE / DONE---")
        return "end"
    # Otherwise there is a function call, so we continue
    else:
        print("---DECISION: RETRIEVE---")
        return "continue"

In [72]:
def grade_documents(state: Type[AgentState]):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (messages): The current state

    Returns:
        str: A decision for whether the documents are relevant or not
    """


    llm = get_giga()

    # Prompt
    prompt = PromptTemplate(
        template="""Вы — оценщик, оценивающий соответствие полученного документа вопросу пользователя. \п
         Вот полученный документ: \n\n {context} \n\n
         Вот вопрос пользователя: {question} \n
         Если документ содержит ключевые слова или семантическое значение, связанное с вопросом пользователя, оцените его как релевантный. \п
         Дайте двоичную оценку "yes" или "no", чтобы указать, имеет ли документ отношение к вопросу.""",
        input_variables=["context", "question"],
    )

    # Chain
    #chain = prompt | llm_with_tool | parser_tool
    chain = prompt | llm

    
    messages = state["messages"]
    last_message = messages[-1]

    print(f"---CHECKING RELEVANCE FOR DOCS--- last_message={last_message}")

    question = messages[0].content

    docs = last_message.content
    
    score = chain.invoke(
        {"question": question, 
         "context": docs}
    )
    print(f"---SCORE--- {score}")
    

    grade = score.content

    if grade == "yes":
        print("---DECISION: DOCS RELEVANT---")
        return "yes"
    else:
        print("---DECISION: DOCS NOT RELEVANT---")
        print(grade)
        return "no"

In [73]:
### Nodes


def agent(state):
    """
    Invokes the agent model to generate a response based on the current state. Given
    the question, it will decide to retrieve using the retriever tool, or simply end.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with the agent response apended to messages
    """
    print("---CALL AGENT---")
    messages = state["messages"]
    
    print("Agent received:", messages)
    model = get_giga()
    
    functions = [convert_to_gigachat_function(t) for t in tools]
    model = model.bind_functions(functions)

    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    
    """
    ----- # due to current code at langchain_community/chat_models/gigachat.py def _convert_dict_to_message(message: Messages) -> BaseMessage:
        if not isinstance(response, BaseMessage):
        raise Exception("Last message is not a BaseMessage instance")
    """


    # If there is no function call, then we finish
    if "function_call" in response.additional_kwargs: 
        if function_call := response.additional_kwargs["function_call"]:
            from gigachat.models.function_call import FunctionCall
            if isinstance(function_call, FunctionCall): 
                response.additional_kwargs["function_call"] = dict(function_call) 
    """
    ----- # due to current code at langchain_community/chat_models/gigachat.py def _convert_dict_to_message(message: Messages) -> BaseMessage:
    """

    return {"messages": [response]}

def retrieve(state):
    """
    Uses tool to execute retrieval.

    Args:
        state (messages): The current state

    Returns:
        dict: The updated state with retrieved docs
    """
    print("---EXECUTE RETRIEVAL---")
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call

    if function_call := last_message.additional_kwargs["function_call"]:
        from gigachat.models.function_call import FunctionCall
        if isinstance(function_call, FunctionCall): 
            # due to current code at langchain_community/chat_models/gigachat.py def _convert_dict_to_message(message: Messages) -> BaseMessage:
            last_message.additional_kwargs["function_call"] = dict(function_call) 

    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=last_message.additional_kwargs["function_call"]["arguments"]
    )

    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)

    function_message = FunctionMessage(content=str(response), name=action.tool)

    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

def rewrite(state):
    """
    Transform the query to produce a better question.
    
    Args:
        state (messages): The current state
    
    Returns:
        dict: The updated state with re-phrased question
    """
    
    print("---TRANSFORM QUERY---")
    messages = state["messages"]
    
    question = messages[0].content

    print(f"---TRANSFORM QUERY---messages={messages}\n-----\n question={question}")

    msg = [HumanMessage(
        content=f""" \n 
        Пользователь ищет информацию в научных статьях. Пользователь формулирует поисковые вопросы к базе знаний.\n 
        Здесь изначальный вопрос пользователя:
        \n ------- \n
        {question} 
        \n ------- \n
        Перефразируй изначальный вопрос пользователя так, чтобы вопрос был полным, завершенным и точным""",
        )]

    # Grader
    model = get_giga()
    response = model.invoke(msg)
     
    print("---TRANSFROM QUERY RESPONSE--- response =", response)

    return {"messages": [response]}

def generate(state):
    """
    Generate answer

    Args:
        state (messages): The current state

    Returns:
         dict: The updated state with re-phrased question
    """
    print("---GENERATE---")
    messages = state["messages"]


    question = messages[0].content

    # LLM
    llm = get_giga()

    # Prompt
    prompt = PromptTemplate(
        template=""", Ты ассистент, который отвечает на вопрос пользователя. 
            Ответь на вопрос пользователя исходя из контекста.
            Если ты не знаешь ответа, просто скажи, что не знаешь, не пытайся придумать ответ. \п
            Вот полученный контекст: \n\n {context} \n\n
            Вот вопрос пользователя: {question} \n
            для ответа на вопрос
            """,
        input_variables=["context", "question"],
    )

    # Chain
    chain = prompt | llm

    response = chain.invoke(
        {"question": question, 
            "context": messages}
)
    return {"messages": [response]}

## Graph

* Start with an agent, `call_model`
* Agent make a decision to call a function
* If so, then `action` to call tool (retriever)
* Then call agent with the tool output added to messages (`state`)

In [74]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes we will cycle between
workflow.add_node("agent", agent)  # agent
workflow.add_node("retrieve", retrieve)  # retrieval
workflow.add_node("rewrite", rewrite)  # retrieval
workflow.add_node("generate", generate)  # retrieval

In [75]:
# Call agent node to decide to retrieve or not
workflow.set_entry_point("agent")

# Decide whether to retrieve
workflow.add_conditional_edges(
    "agent",
    # Assess agent decision
    should_retrieve,
    {
        # Call tool node
        "continue": "retrieve",
        "end": END,
    },
)

# Edges taken after the `action` node is called.
workflow.add_conditional_edges(
    "retrieve",
    # Assess agent decision
    grade_documents,
    {
        "yes": "generate",
        "no": "rewrite",  
    },
)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")

# Compile
app = workflow.compile()

In [76]:
import pprint

from langchain_core.messages import HumanMessage


def qna(question):
    inputs = {
        "messages": [
            HumanMessage(
                content=question
            )
        ]
    }
    for output in app.stream(inputs):
        for key, value in output.items():
            pprint.pprint(f"Output from node '{key}':")
            pprint.pprint("---")
            pprint.pprint(value, indent=2, width=80, depth=None)
        pprint.pprint("\n---\n")

## QnA, когда нужен дополнительный поиск

In [77]:
qna("Какие определения агентов приводятся в статьях на Хабре?")

---CALL AGENT---
Agent received: [HumanMessage(content='Какие определения агентов приводятся в статьях на Хабре?')]


Giga generation stopped with reason: function_call


"Output from node 'agent':"
'---'
{ 'messages': [ AIMessage(content='', additional_kwargs={'function_call': {'name': 'retrieve_blog_posts', 'arguments': {'query': 'определение агент'}}})]}
'\n---\n'
---DECIDE TO RETRIEVE---
---DECISION: RETRIEVE---
---EXECUTE RETRIEVAL---
"Output from node 'retrieve':"
'---'
{ 'messages': [ FunctionMessage(content="Билл Гейтс говорит, что агенты — это тип программного обеспечения, который реагирует на естественный язык и может выполнять множество различных задач на основе знаний пользователя.\xa0Я бы дополнил определение агентов следующим тезисом:«Агент – это программа, которая способна взаимодействовать с внешней средой с помощью инструментов и корректировать своё поведение в зависимости от результатов\n\nи всем распорядком на земле без точного плана на некоторый срок.'}AI-агентыВ завершение хотелось бы немного рассказать про агентов. Все о них говорят, но однозначного понимания того, что такое агент, пока нет. Например, вот так выглядят определения о

## QnA, когда дополнительный поиск не требуется

In [78]:
qna("Что такое ИИ-агент?")

---CALL AGENT---
Agent received: [HumanMessage(content='Что такое ИИ-агент?')]
"Output from node 'agent':"
'---'
{ 'messages': [ AIMessage(content='ИИ-агент (ИИ — искусственный интеллект) — это компьютерная программа, которая может обучаться и принимать решения подобно человеку.')]}
'\n---\n'
---DECIDE TO RETRIEVE---
---DECISION: DO NOT RETRIEVE / DONE---
"Output from node '__end__':"
'---'
{ 'messages': [ HumanMessage(content='Что такое ИИ-агент?'),
                AIMessage(content='ИИ-агент (ИИ — искусственный интеллект) — это компьютерная программа, которая может обучаться и принимать решения подобно человеку.')]}
'\n---\n'
